# Обучение рекуррентной нейронной сети для предсказания следующей фонемы
## В данном коде мы будем обучать базовую рекуррентную нейронную сеть с прямым и обратным распространением

In [273]:
# импорт библиотек
import sys
import numpy as np 
from collections import Counter
import random
import math
import os

In [274]:
# чтение датасета, состоящего из фонетически транскрибированных слов (каждая фонема через пробел)
f = open('phon.txt','r')
raw = f.readlines()
f.close()

# создание пустого списка для токенов
tokens = list()
for line in raw[0:]:  
    tokens.append(line.lower().replace("\n","").split(" ")[0:]) #добавление слов в список токенов

# таким образом токены состоят из списков, содержащих слово в транскрипции 
print(tokens[0:5]) #вывод первых пяти элементов (токенов)
print(len(tokens)) #длина списка токенов

[['j', 'u', "r'", 'i'], ['t', "r'", 'i', 'f', 'a', 'n', 'a', 'f'], ['a', "b'", "m'", 'e', 'n'], ['y', 'j', 'u', "l'", 'e'], ['m', 'a', 'c']]
75235


# Формируем список существующих элементов из токенов и присваиваем им уникальный индекс

In [275]:
vocab = set() #создается простой список фонем из словаря
for sound in tokens: #итерация по каждому элементу в списке 'tokens'
    for phon in sound: #теперь по каждому звуку
        if phon == '': #проверям не пуста ли фонема, если да пропускаем
            pass
        else:
            vocab.add(phon) #добавляем фонему в 'vocab', если фонема уже есть, добавление не происходит
vocab = list(vocab) #преобразование множества в список
phon2index = {} #создание пустого словаря для хранения фонемы и ее индекса
for i,phon in enumerate(vocab):  #итерация по списку 'vocab', получая индекс i и фонему 'phon' 
    phon2index[phon]=i
    
def phons2indices(word, phon2index):
    '''Функция для преобразования списка Фонем в список индексов'''
    idx = list() #список для индексов
    for phon in word: #проходим по фонемам в слове
        if phon == '': #проверяем на пустую строчку
            pass
        else:
            idx.append(phon2index[phon]) #ищем индекс фонемы в словаре 'phon2index' и добавляет его в список 'idx'
    return idx

def softmax(x): 
    '''Функция активации softmax для предсказания следующей фонемы'''
    e_x = np.exp(x - np.max(x)) #ищем экспоненту каждого элемента вектора, вычитая максимальное значение 
    return e_x / e_x.sum(axis=0) #нормализуем экспоненциальные значения, деля на их сумму
    # возвращает вектор вероятностей

# выводим наш список vocab со всеми упомянутыми фонемами
print(vocab)
# длина списка vocab
print(len(vocab))
# выводим словарь, где каждой фонеме присвоен уникальный индекс (начиная с 0)
print(phon2index)
# длина словаря
print(len(phon2index))

["n'", 'zh', "j'", 'm', "v'", "f'", 'd', "t'", 't', 'j', "l'", 'v', "c'", 'e', "sh'", "zh'", 'r', 'sch', 'k', "p'", 'y', "r'", 'b', 'g', 's', 'i', 'c', 'l', 'p', "g'", 'sh', 'ch_', "s'", 'o', "d'", 'dzh', 'sc', 'n', "k'", 'ch', "m'", 'h', 'u', 'f', 'a', "dz'", "b'", "z'", "h'", 'z']
50
{"n'": 0, 'zh': 1, "j'": 2, 'm': 3, "v'": 4, "f'": 5, 'd': 6, "t'": 7, 't': 8, 'j': 9, "l'": 10, 'v': 11, "c'": 12, 'e': 13, "sh'": 14, "zh'": 15, 'r': 16, 'sch': 17, 'k': 18, "p'": 19, 'y': 20, "r'": 21, 'b': 22, 'g': 23, 's': 24, 'i': 25, 'c': 26, 'l': 27, 'p': 28, "g'": 29, 'sh': 30, 'ch_': 31, "s'": 32, 'o': 33, "d'": 34, 'dzh': 35, 'sc': 36, 'n': 37, "k'": 38, 'ch': 39, "m'": 40, 'h': 41, 'u': 42, 'f': 43, 'a': 44, "dz'": 45, "b'": 46, "z'": 47, "h'": 48, 'z': 49}
50


# Разделение данных на обучающую и тестовую выборки 80/20


In [286]:
split_index = int(len(tokens) * 0.8)
train_tokens = tokens[:split_index]
random.shuffle(train_tokens)
test_tokens = tokens[split_index:]
random.shuffle(test_tokens)

In [287]:
# выводим первые 10 токенов обучающей и тестовой выборки 
print(train_tokens[:10])
print(test_tokens[:10]) 

[['p', 'a', "n'", 'i', 'm', 'a', 'e', 'sh'], ['s', 'g', 'u', 'sc', 'a', 'l', 'y', "s'"], ["d'", 'i', 'v', 'a', 'n', 'a'], ['k', 'a', 'z', 'a', 'l'], ['h', 'a', 'l', 'a', "d'", 'i', "l'", "n'", 'i', 'k'], ["s'", 'e', 'k', 'u', 'n', 'd', 'u'], ['k', "r'", 'i', 'm', "l'", 'o', 'f', 's', "k'", 'i', 'i'], ['p', 'a', 'l', 'u', 'ch', 'i', 't'], ["l'", 'e', 't', 'a', 'm'], ['m', 'o', 'sh', 'y']]
[['s', 't', 'r', 'a', 'n', 'y', 'm'], ['e', 'i', 'm', 'u'], ['v', 'y', "t'", 'a', "g'", 'i', 'v', 'a', 'e'], ['p', 'a', 'd', 'n', 'o', 's'], ['n', 'a', 'sh'], ["n'", 'e', 'c'], ['r', 'y', 'z', 'v', 'a', 'r', 'a', 'ch', 'i', 'v', "t'"], ['sh', 't'], ["n'", 'e', 'o', ''], ["v'", 'e', 'r', 'n', 'a']]


# Задаем параметры модели

In [289]:
embed_size = 12 #размерность векторного представления фонем, то есть фонем из 50 элементов
learning_rate = 0.00001 #устанавливаем скорость обучения
BATCH_SIZE = 32 #размер батча
EPOCHS = 100 #количество эпох
dropout_rate = 0.2

# Функция для создания батчей

In [288]:
def create_batches(tokens, batch_size, phons2indices):
    '''Функция принимает список токенов (слов) и размер батча'''
    batches = [] #пустой список для батчей
    for i in range(0, len(tokens), batch_size): #итерирация по списку токенов с шагом, равным размеру батча
        batch = tokens[i:i + batch_size] #извлекаем из списка токена батч
        # фильтруем пустые слова и преобразуем в индексы
        batch = [[phons2indices[token] for token in word if token] for word in batch if word] # вложенный генератор списка
        # фильтруем пустые списки (слова)
        batch = [word for word in batch if word]
        batches.append(batch) #добавляем батч
    return batches

# Матрицы для предсказания
# Инициализируем веса с Xavier


In [239]:
'''Создаем матрицу эмбеддингов (векторных представлений) для фонем. 
Инициализируем матрицу случайными значениями из нормального распределения со средним значением 0 и стандартным отклонением, 
рассчитанным на основе размера словаря фонем размер матрицы = (количество фонем в словаре, размер эмбеддинга)'''
embed = np.random.normal(0, 1/np.sqrt(len(vocab)), size=(len(vocab), embed_size))

'''Создаем рекуррентную матрицу, инициализируем матрицу случайными значениями из нормального распределения со средним значением 0
и стандартным отклонением, рассчитанным на основе размера эмбеддинга размер матрицы = (размер эмбеддинга, размер эмбеддинга)'''
recurrent = np.random.normal(0, 1/np.sqrt(embed_size), size=(embed_size, embed_size))

'''Создаем начальный вектор скрытого состояния и инициализируем вектор нулями размер вектора = (размер эмбеддинга)'''
start = np.zeros(embed_size)

'''Создаем матрицу декодера. Инициализируем матрицу случайными значениями из нормального распределения со средним значением 0
и стандартным отклонением, рассчитанным на основе размера эмбеддинга размер матрицы = (размер эмбеддинга, количество фонем в словаре)'''
decoder = np.random.normal(0, 1/np.sqrt(embed_size), size=(embed_size, len(vocab)))

'''Создаем матрицу one-hot encoding. Создаем единичную матрицу, которая используется для представления фонем в виде one-hot вектора
размер матрицы = (количество фонем в словаре, количество фонем в словаре)'''
one_hot = np.eye(len(vocab))


print('Вектор эмбейдинга:', embed)
print('Матрица декодера:', decoder)

Вектор эмбейдинга: [[ 0.22217504  0.03158586  0.09179856 -0.1710558   0.12330979  0.01854156
   0.27043926  0.08162412 -0.24522745 -0.01043713  0.03181435 -0.13991637]
 [-0.16771883 -0.01625454 -0.06915435  0.03534646  0.16146157 -0.0330332
   0.16403574  0.05446619 -0.33541509 -0.19279283 -0.05290679 -0.03043818]
 [ 0.22302484 -0.31246977  0.06511925  0.12313562 -0.06024685 -0.24229842
   0.12216036  0.0618876  -0.1904398  -0.22902078 -0.18409679  0.10996646]
 [-0.03284311  0.16854042 -0.06562262 -0.0682751  -0.10141138 -0.11307056
  -0.19348044  0.11801743  0.15502453 -0.28761456  0.16508476 -0.13073577]
 [ 0.08022746  0.05130583 -0.08224335 -0.14908136 -0.11711028  0.22144012
   0.06968157 -0.19630513  0.12697555  0.21827534 -0.10495836 -0.02214608]
 [-0.165051    0.16314869  0.05663299  0.07092673  0.02366165  0.12248547
  -0.06930872 -0.02546446 -0.19903565  0.13631828  0.09523311  0.02455648]
 [-0.04008526  0.10813775  0.01142908  0.10996195 -0.03532518  0.06647279
   0.2011956  

# Функция предсказания

In [290]:
# функция активации 
def tanh(x):
    return np.tanh(x)
# функция активации для обратного распространения
def tanh_deriv(x):
    return 1.0 - np.tanh(x)**2

def predict(word, start, recurrent, decoder, embed, one_hot, dropout_rate = 0.0):
    '''Функция принимает на вход список индексов фонем (слова)'''
    layers = list() #Список с именем layers для прямого распространения
    layer = {} #словарб для слоёв
    layer['hidden'] = start #скрытое состояние первого слоя начальным вектором 'start'
    layers.append(layer) #добавляем первый слой в список
    loss = 0 #для хранения суммарной ошибки
    correct_predictions = 0 #правильные предсказания
    total_predictions = 0 #все предсказания

    # итерация по слову
    for target_i in range(len(word)): #проходим по всем фонемам в слове
        layer = {} #словарь для этого слоя
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder)) #извлекает вероятность, которую сеть предсказала для следующей фонемы

        # вычисляем ошибку для текущего предсказания, используя отрицательный логарифм вероятности правильного слова, добавляем ошибку к общей суммарной ошибке 'loss'
        loss += -np.log(layer['pred'][word[target_i]]) #чем ниже вероятность предсказанного слова, тем выше значение loss

        # вычисляем скрытое состояние с использованием функции активации tanh
        hidden = tanh(layers[-1]['hidden'].dot(recurrent) + embed[word[target_i]])

        # drop_out
        # dropout_rate - это вероятность отбрасывания нейрона
        if dropout_rate > 0: #если dropout больше нуля то он применяется
            # создаем бинарную маску как у скрытого слоя и имеет элементы равные единице с вероятностью 1 минус dropout_rate и 0 с вероятностью dropout_rate
            # некоторые нейроны будут умножены на 0 (выключены), а остальные будут умножены на 1 (включены)
            # для обратного распространения градиента, делим маску на (1 - dropout_rate)
            # это приводит к тому что суммарное значение после dropout, такое же как и до dropout
            dropout_mask = np.random.binomial(1, 1 - dropout_rate, size=hidden.shape) / (1 - dropout_rate)
            # применяем его к скрытому слою умножая на маску
            hidden_dropout = hidden * dropout_mask
        else:
            # если dropout = 0, то маска не применяется 
            hidden_dropout = hidden

        # записываем результаты в скрытый слой и скрытый слой с dropout
        layer['hidden'] = hidden
        layer['hidden_dropout'] = hidden_dropout
        layers.append(layer) 

        predicted_index = np.argmax(layer['pred']) #предсказываем индекс следующей фонемы
        if predicted_index == word[target_i]: #если предсказанная фонема совпадает с правильной, то увеличиваем количество верных предсказаний
            correct_predictions += 1
        total_predictions += 1 #увеличиваем общее количество предсказаний

    # вычисляем точность модели
    # делим количество верных предсказаний на общее количество предсказаний
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    # возвращаем результат (слои потерю и точность)
    return layers, loss, accuracy

## Создаем папку для сохранения весов молели

In [241]:
# Путь для сохранения весов модели
save_path = "weights"
if not os.path.exists(save_path):
    os.makedirs(save_path) #если папки нет создаем 
# Переменные для хранения лучших результатов модели
best_accuracy = 0.0 #
best_loss = float('inf')

# Обучение модели

In [242]:
for epoch in range(EPOCHS):
    # создаем батчи из обучающих данных
    train_batches = create_batches(train_tokens, BATCH_SIZE, phon2index)
   
    # задаем переменню общей ошибки, точности, слов 
    total_loss = 0
    total_accuracy = 0
    total_words = 0 

    for batch in train_batches: #цикл для бача на обучающей выборке (задаем все по нулям)
        batch_loss = 0 
        batch_accuracy = 0 
        batch_words = 0
        
        for word in batch: #цикл для каждого слова в батче
            '''Получаем результаты предсказания модели:
            layers: список слоев, содержащих предсказанные значения и другие данные
            loss: значение ошибки для данного слова
            accuracy: значение точности для данного слова'''
            layers, loss, accuracy = predict(word, start, recurrent, decoder, embed, one_hot, dropout_rate=0.2)            
            batch_loss += loss
            batch_accuracy += accuracy
            batch_words += 1
                
            # обратное распространение для обновления весов (вычисляем градиенты ошибки и используем их для корректировки весов модели)
            for layer_idx in reversed(range(len(layers))):
                # итерируем по слоям в обратном порядке
                layer = layers[layer_idx] 
                # получаем текущий слой
                target = word[layer_idx-1]

                # проверяем, что слой не первый (входной)
                if(layer_idx > 0): 
                    layer['output_delta'] = layer['pred'] - one_hot[target] #вычисляем разницу между предсказанным и ожидаемым вектором
                    new_hidden_delta = layer['output_delta'].dot(decoder.transpose()) #вычисляем ошибку скрытого состояния текущего слоя (транспонируя матрицу весов decoder и перемножая с дельтой выходного слоя)
                    
                    # проверяем, что слой не последний (выходной)
                    if(layer_idx == len(layers)-1):
                        layer['hidden_delta'] = new_hidden_delta
                    else:
                        # ошибка скрытого состояния вычисляется на основе ошибки текущего слоя и ошибки скрытого состояния следующего слоя
                        layer['hidden_delta'] = (new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())) * tanh_deriv(layers[layer_idx]['hidden'])
                else: #если слой первый
                    # вычисляем ошибку скрытого состояния первого слоя на основе ошибки скрытого состояния второго слоя
                    layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose()) * tanh_deriv(layers[layer_idx]['hidden'])

            # Ограничение градиента (для предотвращения градиентного взрыва)
            for layer in layers: #цикл по слоям
                if 'hidden_delta' in layer: #проверяем, есть ли ошибка скрытого слоя в текущем слое
                    grad_norm = np.linalg.norm(layer['hidden_delta']) # вычисляем норму градиента
                    if grad_norm > 10: #устанавливаем максимальную норму
                        layer['hidden_delta'] = layer['hidden_delta'] * (10 / grad_norm) # нормализуем градиент если он большой

            '''теперь корректируем веса'''
            # обновляем начальное скрытое состояние на основе ошибки скрытого состояния первого слоя, деля на длину предложения для нормализации
            # вычитаем из начального скрытого состояния (start) произведение ошибки скрытого состояния первого слоя, скорости обучения и деленное на длину предложения.
            start -= layers[0]['hidden_delta'] * learning_rate / (len(word))
            # итерация по всем слоям со второго 
            for layer_idx,layer in enumerate(layers[1:]):
                # корректируем матрицу декодера с учетом скрытого состояния текущего слоя и ошибки выходного слоя, деля на длину слова для нормализации
                decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta']) * learning_rate / (len(word))
                # извлекаем индекс текущего слова из списка индексов предложения
                embed_idx = word[layer_idx]
                # обновляем вектор эмбеддинга текущего слова (embed[embed_idx]), вычитая из текущего значения вектора произведение ошибки скрытого состояния, скорости обучения и деленное на длину слова
                # корректирует вектор эмбеддинга для уменьшения ошибки
                embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * learning_rate / (len(word)) #корректируем вектор эмбеддинга текущего слова с помощью ошибки скрытого состояния текущего слоя, деля на длину предложения для нормализации
                # обновляем матрицу рекуррентной связи, вычитая из текущего значения матрицы внешнее произведение скрытого состояния и ошибки скрытого состояния, умноженное на скорость обучения и деленное на длину предложения
                # корректирует матрицу рекуррентной связи для уменьшения ошибки при обновлении скрытого состояния
                recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * learning_rate / (len(word)) #обновляем матрицу рекуррентной связи с помощью скрытого состояния текущего слоя и ошибки скрытого состояния текущего слоя, деля на длину предложения для нормализации
            
        if batch_words > 0: #проверка на пустые батчи
            total_loss += batch_loss #суммируем ошибки для всего батча
            total_accuracy += batch_accuracy #суммируем точность для всего батча
            total_words += batch_words #суммируем количество слов в батче
    if total_words > 0: #проверяем было ли обработано хоть одно слово в текущей эпохе
         avg_loss = total_loss/total_words #среднее значение ошибки
         avg_accuracy = total_accuracy/total_words #среднее значение точности
    else: #если в эпохе не было обработано слов, то средние значения равны нулю
         avg_loss = 0
         avg_accuracy = 0

    # оценка модели на тестовой выборке
    test_accuracy = 0
    test_words = 0
    test_loss = 0
    
    # создаем батчи для тестовой выборки
    test_batches = create_batches(train_tokens, BATCH_SIZE, phon2index)
    # цикл по тестовым батчам 
    for test_batch in test_batches:
        batch_test_accuracy = 0 #переменная для хранения точности текущего тестового батча
        batch_test_loss = 0 #переменная для хранения потери текущего тестового батча
        batch_test_words = 0 #переменняа для хранения количсетва слов в тестовом батче

        # цикл по всем словам в тестовом батче
        for word in test_batch:
            layers, loss, accuracy = predict(word, start, recurrent, decoder, embed, one_hot, dropout_rate=0.2)            
            batch_test_accuracy += accuracy #добавляем точность 
            batch_test_loss += loss #добавляем потерю
            batch_test_words += 1 #бобавляем счет слов

        # проверка на наличие хотя бы одного слова в батче и так же суммируем
        if batch_test_words > 0:
            test_accuracy += batch_test_accuracy 
            test_loss += batch_test_loss
            test_words += batch_test_words

    # проверяем, было ли обработано хотя бы одно слово в тестовой выборке
    if test_words > 0:
        avg_test_accuracy = test_accuracy/test_words #вычисляем среднюю точность, деля тестовую точность на количество слов
        avg_test_loss = test_loss/test_words #вычисляем среднюю ошибку, деля тестовую ошибку на количество слов
    # если не было обработано ни одного слова средние значения 0
    else:
        avg_test_accuracy = 0
        avg_test_loss = 0
          
    # вывод процесса обучения
    print(f"Эпоха {epoch + 1}/{EPOCHS}, "
          f"Обучение: Loss: {avg_loss:.3f}, Accuracy: {avg_accuracy:.3f} , "
          f"Тест: Loss: {avg_test_loss:.3f}, Accuracy: {avg_test_accuracy:.3f}")
    # прерываем обучение если среднняя ошибка nan 
    if np.isnan(avg_loss): 
        print('Ошибка: nan')
        break  
    
    # проверка на улучшение
    # ниже мы проверяем улучшилась ли модель от эпохи к эпохе 
    # устанавливая условие - либо текущая точность на тестовой выборке (avg_test_accuracy) больше лучшей предыдущей (best_accuracy), 
    # либо текущая точность равна лучшей, но текущая ошибка на тестовой выборке (avg_test_loss) меньше лучшей предыдущей (best_loss)
    if avg_test_accuracy > best_accuracy or (avg_test_accuracy == best_accuracy and avg_test_loss < best_loss):
            # обновляем лучшую точность на ту, которую получили по условию
            best_accuracy = avg_test_accuracy
            # так же с потерей
            best_loss = avg_test_loss
            # сохранение весов в нашу папку для весов
            np.save(os.path.join(save_path, "embed.npy"), embed)
            np.save(os.path.join(save_path, "recurrent.npy"), recurrent)
            np.save(os.path.join(save_path, "start.npy"), start)
            np.save(os.path.join(save_path, "decoder.npy"), decoder)
            print('Веса обновлены')

Эпоха 1/100, Обучение: Loss: 21.434, Accuracy: 0.018 , Тест: Loss: 21.415, Accuracy: 0.021
Веса обновлены
Эпоха 2/100, Обучение: Loss: 21.399, Accuracy: 0.025 , Тест: Loss: 21.381, Accuracy: 0.029
Веса обновлены
Эпоха 3/100, Обучение: Loss: 21.365, Accuracy: 0.029 , Тест: Loss: 21.348, Accuracy: 0.030
Веса обновлены
Эпоха 4/100, Обучение: Loss: 21.330, Accuracy: 0.032 , Тест: Loss: 21.311, Accuracy: 0.034
Веса обновлены
Эпоха 5/100, Обучение: Loss: 21.290, Accuracy: 0.036 , Тест: Loss: 21.267, Accuracy: 0.038
Веса обновлены
Эпоха 6/100, Обучение: Loss: 21.240, Accuracy: 0.041 , Тест: Loss: 21.210, Accuracy: 0.048
Веса обновлены
Эпоха 7/100, Обучение: Loss: 21.173, Accuracy: 0.051 , Тест: Loss: 21.132, Accuracy: 0.056
Веса обновлены
Эпоха 8/100, Обучение: Loss: 21.081, Accuracy: 0.063 , Тест: Loss: 21.025, Accuracy: 0.070
Веса обновлены
Эпоха 9/100, Обучение: Loss: 20.957, Accuracy: 0.077 , Тест: Loss: 20.882, Accuracy: 0.081
Веса обновлены
Эпоха 10/100, Обучение: Loss: 20.797, Accuracy

# Попросим сеть предсказать следующую фонему в слове с индексом n

# Загрузка весов модели

In [261]:
def load_model(save_path):
    """Функция для загрузки весов модели"""
    embed = np.load(os.path.join(save_path, "embed.npy"))
    recurrent = np.load(os.path.join(save_path, "recurrent.npy"))
    start = np.load(os.path.join(save_path, "start.npy"))
    decoder = np.load(os.path.join(save_path, "decoder.npy"))
    return embed, recurrent, start, decoder
# определяем веса из загруженных данных
embed, recurrent, start, decoder = load_model(save_path)

In [262]:
word_index = 15
# получаем предсказания модели для слова с индексом word_index, преобразуя его фонемы в числовые индексы
embed, recurrent, start, decoder = load_model(save_path)
l, _, _ = predict(phons2indices(tokens[word_index], phon2index), start, recurrent, decoder, embed, one_hot)
print(tokens[word_index])
# цикл по всем слоям в списке предсказаний (l), кроме входного и выходного слоя 
for i, each_layer in enumerate(l[1:-1]):
    # получаем входную фонему для текущего слоя из оригинального слова
    input_phon = tokens[word_index][i]
    # получаем настоящую фонему для текущего слоя из оригинального слова
    true_phon = tokens[word_index][i + 1]
    # получаем предсказаную фонему для текущего слоя из оригинального слова (argmax() используется для нахождения индекса максимальной вероятности в предсказании)
    pred_phon = vocab[each_layer['pred'].argmax()]
    print(f"Предыдущая фонема: {input_phon} Истинная: {true_phon} Предсказанная: {pred_phon}")

['p', 'r', 'a', "l'", 'i', 'zh', 'e', 'l', 'a']
Предыдущая фонема: p Истинная: r Предсказанная: a
Предыдущая фонема: r Истинная: a Предсказанная: a
Предыдущая фонема: a Истинная: l' Предсказанная: a
Предыдущая фонема: l' Истинная: i Предсказанная: a
Предыдущая фонема: i Истинная: zh Предсказанная: a
Предыдущая фонема: zh Истинная: e Предсказанная: a
Предыдущая фонема: e Истинная: l Предсказанная: a
Предыдущая фонема: l Истинная: a Предсказанная: a


In [263]:
word_index = 25
# получаем предсказания модели для слова с индексом word_index, преобразуя его фонемы в числовые индексы
embed, recurrent, start, decoder = load_model(save_path)
l, _, _ = predict(phons2indices(tokens[word_index], phon2index), start, recurrent, decoder, embed, one_hot)
print(tokens[word_index])
# цикл по всем слоям в списке предсказаний (l), кроме входного и выходного слоя 
for i, each_layer in enumerate(l[1:-1]):
    # получаем входную фонему для текущего слоя из оригинального слова
    input_phon = tokens[word_index][i]
    # получаем настоящую фонему для текущего слоя из оригинального слова
    true_phon = tokens[word_index][i + 1]
    # получаем предсказаную фонему для текущего слоя из оригинального слова (argmax() используется для нахождения индекса максимальной вероятности в предсказании)
    pred_phon = vocab[each_layer['pred'].argmax()]
    print(f"Предыдущая фонема: {input_phon} Истинная: {true_phon} Предсказанная: {pred_phon}")

['p', 'a', 't', "v'", 'e', 'r', "d'", 'i', 'l', 'a', "s'"]
Предыдущая фонема: p Истинная: a Предсказанная: a
Предыдущая фонема: a Истинная: t Предсказанная: a
Предыдущая фонема: t Истинная: v' Предсказанная: a
Предыдущая фонема: v' Истинная: e Предсказанная: a
Предыдущая фонема: e Истинная: r Предсказанная: a
Предыдущая фонема: r Истинная: d' Предсказанная: a
Предыдущая фонема: d' Истинная: i Предсказанная: a
Предыдущая фонема: i Истинная: l Предсказанная: a
Предыдущая фонема: l Истинная: a Предсказанная: a
Предыдущая фонема: a Истинная: s' Предсказанная: a
